# Best Place to Operate a Japanese restaurant in Downtown Toronto

### Introduction: 
#### Toronto is the largest city in Canada with population (2016) 2,731,571. And Downtown Toronto is the heart of the city. It has Union Station, headquarters of all big five banks of Canada as well as stadiums of NBA and NHL teams. This project is aim to help well represented franchise Japanese restaurants finding the best place for their restaurant expansion in Downtown Toronto area of city of Toronto. This project will be using k-cluster methodology to find the best neighborhood by considering various factors. 

#### Location is critical for restaurants. Even good food but with bad location will still hurt the business. However, with so many neighborhoods in a city, it is very time consuming to look into each neighborhood with many different factors like traffic, competitors, scenic spots and so on. Therefore, using k- means clustering is a very good way for finding the top spots and for decision making as well.  

### About the Data:

#### The data for this project will be the same as the project from Week 3. The first source is "List of postal codes of Canada: M". It has three columns: Postal code, Borough, Neighbourhood. There are total 16 different neighborhoods in Downtown Toronto area. And the second source is "https://cocl.us/Geospatial_data" which contains the latitude and longitube of each postal code. The last is Foursqure location data. The methodology will be k-means clustering.

#### Below are some key factors we consider to apply on the results after the clustering:
##### 1. A neighborhood that already has some restaurants. This makes sure that the neighborhood is kind of known for food. 
##### 2. However, we want less similar restaurants in the neighborhood. It is to avoid too much competition at the beginning and also to make the restaurant unique. 
##### 3. Public transportation. There will be subway and bus stops around. 
##### 3. Some popular spots around like museums, shopping centers, theatres and hotels. This makes sure the customer traffic including local residents and tourists. 

### Methodology

#### First, I want to bring in and combined the first two data that I mentioned above: "List of postal codes of Canada: M" and the geo file.

In [38]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

new_df = df[~df["Borough"].str.contains('Not assigned')] #ignore rows with 'Not assigned'

sdf = new_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index() # Merge duplicated postcode

sdf['Neighbourhood'] = np.where(sdf['Neighbourhood'].str.contains('Not assigned'), sdf["Borough"], sdf['Neighbourhood']) #Replace "Not assigned" in "Neibourhood" column by value in "Borough" column

sdf2 = pd.read_csv('https://cocl.us/Geospatial_data') #Load geo file
    
sdf.loc[sdf['Postcode'] == sdf2['Postal Code'], 'Latitude'] = sdf2['Latitude']  #Add latitude column

sdf.loc[sdf['Postcode'] == sdf2['Postal Code'], 'Longitude'] = sdf2['Longitude']  #Add longitude column

sdf

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### It now has all the neighborhood and its latitude and longitude line up. We can see there are 101 different neighborhoods in whole city of Toronto with 16 in Downtown Toronto area.

#### Next, I want to bring in Foursquare API.

In [2]:
CLIENT_ID = 'CHWNFC4U3HZBMSGT4ZWYI0E23POV02MPC0XYE304OA4ZXQQW' # your Foursquare ID
CLIENT_SECRET = 'L0SBPFHP4DZMOMJJMCXYAHALGSPDSXMWS05EQ311YNYU3EHU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CHWNFC4U3HZBMSGT4ZWYI0E23POV02MPC0XYE304OA4ZXQQW
CLIENT_SECRET:L0SBPFHP4DZMOMJJMCXYAHALGSPDSXMWS05EQ311YNYU3EHU


#### Now, we narrow down the area to Downtown Toronto

In [3]:
address = 'Downtown Toronto,ON'

geolocator = Nominatim(user_agent="dtt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude

longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [4]:
dtt_data = sdf[sdf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtt_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


### Print out the map of Donwtown Toronto

In [5]:
map_dtt = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dtt_data['Latitude'], dtt_data['Longitude'], dtt_data['Borough'], dtt_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtt)  
    
map_dtt

In [34]:
neighbourhood_latitude = dtt_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = dtt_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = dtt_data.loc[0, 'Neighbourhood'] # neighborhood name

In [33]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [35]:
results = requests.get(url).json()

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### All the venues with their category in Downtown Toronto

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### List out all neighborhoods in Downtown Toronto.

In [13]:
dtt_venues = getNearbyVenues(names=dtt_data['Neighbourhood'],            # List out all neighborhoods in Downtown Toronto.
                                   latitudes=dtt_data['Latitude'],
                                   longitudes=dtt_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


### Group neighborhoods and their venue counts

In [37]:
dtt_venues.groupby('Neighbourhood').count() #group neighborhoods and their venue counts.

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [30]:
# one hot encoding
dtt_onehot = pd.get_dummies(dtt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtt_onehot['Neighbourhood'] = dtt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dtt_onehot.columns[-1]] + list(dtt_onehot.columns[:-1])
dtt_onehot = dtt_onehot[fixed_columns]

### Print out the frequency per venue category per neighborhood.

In [18]:
dtt_grouped = dtt_onehot.groupby('Neighbourhood').mean().reset_index() #Print out the frequency per venue category per neighborhood.
dtt_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.02,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.010000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.010000,0.010000,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.010000,0.000000,0.00

### Print each neighborhood along with the top 10 most common venues

In [19]:
#Print each neighborhood along with the top 10 most common venues

num_top_venues = 10

for hood in dtt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dtt_grouped[dtt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.04
2           Steakhouse  0.04
3                  Bar  0.04
4      Thai Restaurant  0.04
5  American Restaurant  0.03
6         Burger Joint  0.03
7     Sushi Restaurant  0.03
8               Bakery  0.03
9           Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3          Steakhouse  0.04
4                 Pub  0.04
5          Restaurant  0.04
6                Café  0.04
7              Bakery  0.04
8         Cheese Shop  0.04
9  Seafood Restaurant  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0     Airport Service  0.21
1      Airport Lounge  0.14
2    Airport Terminal  0.14
3               Plane  0.07
4     Harbor / Marina  0.07
5    Sculpture Garden  0.07
6    

### Create the new dataframe and display the top 10 venues for each neighborhood.

In [20]:
#create the new dataframe and display the top 10 venues for each neighborhood.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = dtt_grouped['Neighbourhood']

for ind in np.arange(dtt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Thai Restaurant,Café,American Restaurant,Hotel,Sushi Restaurant,Bakery,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Steakhouse,Restaurant,Farmers Market,Café,Pub,Bakery
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Plane,Sculpture Garden,Harbor / Marina,Airport Gate,Airport Food Court,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Indian Restaurant,Bank,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Bar,Burger Joint,Sandwich Place,Sushi Restaurant,Thai Restaurant,Middle Eastern Restaurant


### Check cluster labels generated for each row in the dataframe

In [21]:
# set number of clusters
kclusters = 6

dtt_grouped_clustering = dtt_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtt_grouped_clustering)

# check cluster labels generated for each row in the dataframe

kmeans.labels_

array([4, 4, 3, 4, 4, 5, 2, 4, 4, 4, 4, 5, 0, 0, 1, 4, 4, 4], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [22]:
#Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtt_merged = dtt_data

# merge york_grouped with york_data to add latitude/longitude for each neighborhood
dtt_merged = dtt_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dtt_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,4,Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Indian Restaurant,Bank,Japanese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Bubble Tea Shop
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Dessert Shop,Brewery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,4,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Diner,Italian Restaurant,Lingerie Store,Bubble Tea Shop
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Gastropub,Cosmetics Shop,Italian Restaurant,Bakery,Gym
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Steakhouse,Restaurant,Farmers Market,Café,Pub,Bakery
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Bar,Burger Joint,Sandwich Place,Sushi Restaurant,Thai Restaurant,Middle Eastern Restaurant
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,4,Coffee Shop,Bar,Steakhouse,Thai Restaurant,Café,American Restaurant,Hotel,Sushi Restaurant,Bakery,Restaurant
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Fried Chicken Joint,Scenic Lookout,Bakery,Pizza Place,Brewery


### Map of Downtown Toronto with cluster markers

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13
                        )

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtt_merged['Latitude'], dtt_merged['Longitude'], dtt_merged['Neighbourhood'], dtt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Result:

### Examine clusters

#### Cluster 0

In [24]:
#cluster 0
dtt_merged.loc[dtt_merged['Cluster Labels'] == 0, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Dessert Shop,Brewery
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Fried Chicken Joint,Scenic Lookout,Bakery,Pizza Place,Brewery


#### Cluster 1

In [25]:
#cluster 1
dtt_merged.loc[dtt_merged['Cluster Labels'] == 1, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 2

In [26]:
#cluster 2
dtt_merged.loc[dtt_merged['Cluster Labels'] == 2, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Restaurant,Diner,Baby Store,Italian Restaurant,Nightclub,Convenience Store,Coffee Shop


#### Cluster 3

In [27]:
#cluster 3
dtt_merged.loc[dtt_merged['Cluster Labels'] == 3, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Plane,Sculpture Garden,Harbor / Marina,Airport Gate,Airport Food Court,Airport


#### Cluster 4

In [28]:
#cluster 4
dtt_merged.loc[dtt_merged['Cluster Labels'] == 4, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Indian Restaurant,Bank,Japanese Restaurant
2,Downtown Toronto,4,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Bubble Tea Shop
4,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Diner,Italian Restaurant,Lingerie Store,Bubble Tea Shop
5,Downtown Toronto,4,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Gastropub,Cosmetics Shop,Italian Restaurant,Bakery,Gym
6,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Steakhouse,Restaurant,Farmers Market,Café,Pub,Bakery
7,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Bar,Burger Joint,Sandwich Place,Sushi Restaurant,Thai Restaurant,Middle Eastern Restaurant
8,Downtown Toronto,4,Coffee Shop,Bar,Steakhouse,Thai Restaurant,Café,American Restaurant,Hotel,Sushi Restaurant,Bakery,Restaurant
10,Downtown Toronto,4,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Thai Restaurant,Steakhouse
11,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Deli / Bodega,Italian Restaurant
15,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Pub,Seafood Restaurant,Fast Food Restaurant,Cocktail Bar,Hotel,Art Gallery,Breakfast Spot


#### Cluster 5

In [29]:
#cluster 5
dtt_merged.loc[dtt_merged['Cluster Labels'] == 5, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,5,Café,Japanese Restaurant,Restaurant,Bakery,Bar,Bookstore,Theater,Beer Bar,Comfort Food Restaurant,Sandwich Place
13,Downtown Toronto,5,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Burger Joint,Dessert Shop


## Discussion

### In Downtown Toronto area, public transportations are very seperated from popular venues from other categories. By looking at the result, only cluster 3 has public transportations in its top 10 common venues. But there are no other entertainments spot in cluster 3, like restaurants, cafe, pub etc. And in the rest of the clusters, no public transportation venues in the top 10 common venues. So in this case, I think we can exclude public transportation from our factor list.  

### We want the restarant to be opened in a neighborhood that kind of known for food, so we'd like to see a cluster with more restaurants in top 10 venues. Cluster 4 and cluster 5 seems to meet the requirement. In cluster 4, we see more western restaurants than Asian restaurants. While in cluster 5, we see the both neighborhoods in it has Asian restaurants on their 2nd most common venues. Also, cluster 4 has much more neighboorhoods than any other clusters, and this makes sure that the population and traffic is huge compared to other clusters. And we also want to avoid some competitions from similar category, so cluster 4 seems to meet our requirement. 

### Now we to determine the best neighborhood from cluster hood. We want to exclude neighboorhood No.2, since it has a Japanese restaurant on the 1st most common venue. Neighboorhood No.5 seems to meet most of our requirements: It has no Japanese restaurant but many different kind s of venues like hotel, gym, Italian restaurant and cosmetic shop. It is more diverse compared with other neighborhoods in cluster 4. And this can make sure that there is good amount of traffic but less competition from similar restaurants. 

## Conclusion

### By examining the clusters and applying our requirements in the chosen cluster, we decided to choose "St. James Town" as the location for the restaurant expansion project. This methodology cluster out the neighborhoods and let us easily to apply our requirements on it. This is very helpful on decision making and helpful for business investors to lower the risk of finding bad location. It can also be applied on choosing location for other business from different categories with similar information provided.